### Multi-layer Perceptron
This notebook contains an MLP implementation from scratch without using a deep learning library.\
Only Numpy is used as third-party library for mathematical usage including vectors. \
The source code written by the guadiance of [Neural Networks and Deep Learning](http://neuralnetworksanddeeplearning.com) book written by Michael Nielsen. \
Additionally, the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset is utilized to train and test the model for handwritten digit recognition. 

In [23]:
import random
import numpy as np
import mnist_loader # for loading the MNIST dataset

In [165]:
class MLP(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes) # number of layers
        self.sizes = sizes # size of layers respectively
        self.biases = [np.random.randn(y,1) for y in sizes[1:]] # random initialized biases for each layer except input
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])] # random initialized weights 
    
    """
        Return the output of the network if 'a' is input
    """
    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)
        return a
    
    """ 
        Train the network by using mini-batch Stochastic Gradient Descent 
        First, shuffles the dataset and divides into mini-batches regarding the mini_batch_size arg.
        Then, applies SGD to each batch by calling mini_batch_update func. which updates weights and biases
    """
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        if test_data:
            n_test = len(test_data)
        n = len(training_data)

        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)

            if test_data:
                print(f"Epoch {j} complete: {self.evaluate(test_data)} / {n_test}")
            else:
                print(f"Epoch {j} complete")

    """
        Updates weights and biases by using backpropagation to a single mini batch
    """
    def update_mini_batch(self, mini_batch, eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]

        self.weights = [w - (eta/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b - (eta/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)]
        
    """
        This is where the magic happens, still don't know what's going on here; total black box for me.
        As far as I understood, backpropagation is an expression for the partial derivative of the cost
        function C with respect to any weight or bias in the network.
        The algorithm is taken from the Chapter 2 of the guide book.
        
        Computes the gradient of the cost function for a single training using SGD
        Returns a tuple (nabla_b, nabla_w) representing the gradient for the cost function
    """
    def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = self.sigmoid(z)
            activations.append(activation)

        # backward pass
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose()) 

        # mindf#ck here
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        
        return (nabla_b, nabla_w)
    
    """
        Return the number of test inputs for which the network ouputs the correct result
    """
    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x==y) for (x,y) in test_results)
    
    """
        The sigmoid function
    """
    def sigmoid(self, z):
        return 1.0 / (1.0 + np.exp(-z))

    """
        Derivative of the sigmoid function
    """
    def sigmoid_prime(self, z):
        return sigmoid(z) * (1-sigmoid(z))

    """
        Return the vector of partial derivatives for the output activations
    """
    def cost_derivative(self, output_activations, y):
        return (output_activations - y)

In [166]:
# Loading the dataset into training, validation and test sets
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

In [167]:
# hyper-parameters
learning_rate = 3.0
epochs = 30
mini_batch_size = 10

In [168]:
# Init. the network which has:
# input layer with 784 inputs representing the 28x28 image vectors
# hidden layer with 30 neurons
# output layer with 10 neurons representing the classes

mlp = MLP([784, 30, 10])

In [169]:
mlp.SGD(training_data, epochs, mini_batch_size, learning_rate, test_data=test_data)

Epoch 0 complete: 9064 / 10000
Epoch 1 complete: 9263 / 10000
Epoch 2 complete: 9330 / 10000
Epoch 3 complete: 9400 / 10000
Epoch 4 complete: 9418 / 10000
Epoch 5 complete: 9432 / 10000
Epoch 6 complete: 9414 / 10000
Epoch 7 complete: 9448 / 10000
Epoch 8 complete: 9468 / 10000
Epoch 9 complete: 9479 / 10000
Epoch 10 complete: 9483 / 10000
Epoch 11 complete: 9497 / 10000
Epoch 12 complete: 9520 / 10000
Epoch 13 complete: 9503 / 10000
Epoch 14 complete: 9492 / 10000
Epoch 15 complete: 9518 / 10000
Epoch 16 complete: 9504 / 10000
Epoch 17 complete: 9502 / 10000
Epoch 18 complete: 9505 / 10000
Epoch 19 complete: 9513 / 10000
Epoch 20 complete: 9503 / 10000
Epoch 21 complete: 9499 / 10000
Epoch 22 complete: 9538 / 10000
Epoch 23 complete: 9509 / 10000
Epoch 24 complete: 9523 / 10000
Epoch 25 complete: 9527 / 10000
Epoch 26 complete: 9511 / 10000
Epoch 27 complete: 9533 / 10000
Epoch 28 complete: 9528 / 10000
Epoch 29 complete: 9517 / 10000
